<a href="https://colab.research.google.com/github/Savannah-Stuart10/TaylorSwiftData/blob/Taylor-Swift---Baby-Let-the-Games-Begin!/Finished_Song_Game_All_of_Taylor_Swift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guess The Song - Taylor Swift Edition

Welcome to Guess The Song - Taylor Swift Edition!

This is a game that allows the user to guess words to a random Taylor Swift song in an allotted amount of time.

Guess the song lyrics correctly before the time runs out and you win!

If you want to give up or can't think of the lyric, type 'quit' or 'show'

There are still some edits that need to be made to this game, which includes editing the .txt files directly that this game uses. Unfortunately, this will take some time, but most of the lyrics should be correct.

Special shoutout to @wjakethompson for this raw data. Could not have done this without your .txt files, so thank you and... baby let the games begin 🤘

*You only need to run the first cell once in the same session. If you close out of this session, you will need to run the first cell again*

In [9]:
import requests

def get_github_file_list(standard_github_url, depth=0):
    indent = "  " * depth
    api_url = standard_github_url.replace("https://github.com/", "https://api.github.com/repos/").replace("/tree/main/", "/contents/").replace("/blob/main/", "/contents/")
    response = requests.get(api_url)
    response.raise_for_status()

    file_paths = []
    response_json = response.json()

    if isinstance(response_json, list):
        for item in response_json:
            if item['type'] == 'file':
                if item['name'].endswith('.txt'):
                    file_paths.append(item['path'])
            elif item['type'] == 'dir':
                subdirectory_url = f"https://github.com/wjakethompson/taylor/tree/main/{item['path']}"
                file_paths.extend(get_github_file_list(subdirectory_url, depth + 1))
        return file_paths
    else:
        return []

def get_github_file_content(raw_content_url):
    response = requests.get(raw_content_url)
    response.raise_for_status()
    return response.text

standard_github_base_url = "https://github.com/wjakethompson/taylor/tree/main/data-raw/lyrics"

raw_content_repo_base_url = "https://raw.githubusercontent.com/wjakethompson/taylor/main/"

file_paths = get_github_file_list(standard_github_base_url)

print(f"\nFound {len(file_paths)} .txt files:")

lyrics_data = {}
if file_paths:
    for file_path in file_paths:
        raw_file_url = f"{raw_content_repo_base_url}{file_path}"
        song_title = file_path.split('/')[-1].replace(".txt", "")
        try:
            lyrics = get_github_file_content(raw_file_url)
            lyrics_data[song_title] = lyrics
        except requests.exceptions.RequestException as e:
            print(f"Error fetching lyrics for {file_path}: {e}")

print(f"Loaded lyrics for {len(lyrics_data)} songs.")


Found 364 .txt files:
Loaded lyrics for 364 songs.


Everytime you want to start a new game, you will need to run each of the following cells once.

In [10]:
import random

if lyrics_data:
    random_song_title = random.choice(list(lyrics_data.keys()))

    random_song_lyrics = lyrics_data[random_song_title]

    print(f"Selected song")
else:
    print("No lyrics data loaded to select a song from.")

Selected song


In [ ]:
import re

def prepare_lyrics_for_game(lyrics):

    words = re.findall(r'\b\w+\b', lyrics.lower())

    game_words = []
    blanked_word_map = {}

    for i, word in enumerate(words):
        blank_id = f"BLANK_{i}"
        game_words.append("______")
        blanked_word_map[blank_id] = word


    return game_words, blanked_word_map

if 'random_song_lyrics' in locals():
    game_words, blanked_word_map = prepare_lyrics_for_game(random_song_lyrics)

    print(f"\nNumber of blanks created: {len(blanked_word_map)}")
else:
    print("Please select a song first by running the previous cell.")


Number of blanks created: 304


In [ ]:
import time
import re

def prepare_lyrics_with_lines_for_game(lyrics):
    lines = lyrics.split('\n')
    processed_lines_segments = []
    blanked_word_map = {}
    word_to_blank_ids = {}
    blank_counter = 0

    markers_to_exclude = [
        r'^\[?chorus\]?$',
        r'^\[?verse \d+\]?$',
        r'^\[?bridge\]?$',
        r'^\[?outro\]?$',
        r'^\[?intro\]?$',
        r'^\[?pre-?chorus\]?$',
        r'^\[?instrumental\]?$',
        r'^\[?fade out\]?$'
    ]
    markers_pattern = re.compile('|'.join(markers_to_exclude), re.IGNORECASE)

    word_pattern = re.compile(r'\b\w+\'?\w*\b')

    for line in lines:
        line_segments = []
        last_end = 0
        for match in word_pattern.finditer(line):
            word = match.group(0)
            start, end = match.span()

            if start > last_end:
                line_segments.append(('text', line[last_end:start]))

            if markers_pattern.match(word.lower()):
                line_segments.append(('text', word))
            else:
                blank_id = f"BLANK_{blank_counter}"
                line_segments.append(('blank', blank_id, word.lower()))
                blanked_word_map[blank_id] = word.lower()

                if word.lower() not in word_to_blank_ids:
                    word_to_blank_ids[word.lower()] = []
                word_to_blank_ids[word.lower()].append(blank_id)

                blank_counter += 1

            last_end = end

        if last_end < len(line):
            line_segments.append(('text', line[last_end:]))

        processed_lines_segments.append(line_segments)

    return processed_lines_segments, blanked_word_map, word_to_blank_ids


def play_game(processed_lines_segments, blanked_word_map, word_to_blank_ids, time_limit_seconds=60):
    print("\n--- Fill in the Blanks ---")

    if 'random_song_title' in globals():
        print(f"Guess the lyrics for the song")
    else:
        print("Guess the lyrics for the selected song")

    print(f"You have {time_limit_seconds} seconds to guess the words.")
    print("-" * 25)

    guessed_original_words = set()
    filled_blank_ids = set()

    start_time = time.time()

    print("Enter your guesses for the blanks.")
    print("Type 'quit' to end the game.")
    print("Type 'show' to see the current state of the lyrics.")

    all_original_words_to_guess = set(blanked_word_map.values())

    while len(guessed_original_words) < len(all_original_words_to_guess):
        time_elapsed = time.time() - start_time
        if time_elapsed > time_limit_seconds:
            print("\nTime's up!")
            break

        print("-" * 25)
        for line_segments in processed_lines_segments:
            display_line_parts = []
            for segment_type, value, *rest in line_segments:
                if segment_type == 'text':
                    display_line_parts.append(value)
                elif segment_type == 'blank':
                    blank_id = value
                    original_word = rest[0]
                    if blank_id in filled_blank_ids:
                        display_line_parts.append(original_word.upper())
                    else:
                        display_line_parts.append("______")
            print("".join(display_line_parts))
        print("-" * 25)


        time_left = max(0, int(time_limit_seconds - time_elapsed))
        print(f"Time left: {time_left}s - {len(all_original_words_to_guess) - len(guessed_original_words)} unique words remaining.")
        guess = input("Your guess: ").strip()

        if guess.lower() == 'quit':
            print("Quitting game.")
            break
        elif guess.lower() == 'show':
             continue

        processed_guess_words = re.findall(r'\b\w+\b', guess.lower())
        if not processed_guess_words:
            print("Please enter a word.")
            continue

        guessed_word_lower = guess.lower()

        if guessed_word_lower in word_to_blank_ids:
            if guessed_word_lower in guessed_original_words:
                print(f"You already correctly guessed '{guessed_word_lower}'.")
            else:
                print(f"Correct! Filling in all instances of '{guessed_word_lower}'.")
                blank_ids_to_fill = word_to_blank_ids[guessed_word_lower]

                filled_blank_ids.update(blank_ids_to_fill)

                guessed_original_words.add(guessed_word_lower)

        else:
            print(f"'{guess}' is not one of the blanked words.")

    print("\n--- Final Lyrics ---")
    for line_segments in processed_lines_segments:
        display_line_parts = []
        for segment_type, value, *rest in line_segments:
            if segment_type == 'text':
                display_line_parts.append(value)
            elif segment_type == 'blank':
                blank_id = value
                original_word = rest[0]
                if blank_id in filled_blank_ids:
                     display_line_parts.append(original_word.upper())
                else:
                     display_line_parts.append(original_word)
        print("".join(display_line_parts))
    print("-" * 25)

    if len(guessed_original_words) == len(all_original_words_to_guess):
        print("Congratulations! You filled in all the blanks!")
    else:
        print(f"You guessed {len(guessed_original_words)} out of {len(all_original_words_to_guess)} unique words correctly.")

if 'random_song_lyrics' in locals():
    processed_lines_segments, blanked_word_map, word_to_blank_ids = prepare_lyrics_with_lines_for_game(random_song_lyrics)

    print(f"\nNumber of unique words to guess: {len(set(blanked_word_map.values()))}")

    game_time_limit = 900

    play_game(processed_lines_segments, blanked_word_map, word_to_blank_ids, time_limit_seconds=game_time_limit)

else:
    print("Please select a song and prepare the lyrics for the game first.")

Streaming output truncated to the last 5000 lines.
______ ______ ______ '______ ______ ______ ______ ______ (______)
______ ______ '______ I ______ ______ ______ ______ (______, ______, ______, ______, ______, ______)

[Chorus]
I ______ ______ THINGS, BUT ______ ______ YOU ______ ______ ______
______ ______, ______ RIGHT
______, ______ ______ ______ I WANT, ______
I ______ ______ ______ ______ ______ ______ ______ ______ TO ______
______ ______, ______ RIGHT
______, ______ ______ ______ I WANT
______ ______ ______, ______ ______ ______, ______ ______ ______
______, ______ ______ ______ I WANT

[______ ______]
______ ______ ______, ______ ______ ______ ______ ______
______ YOU ______ ______ ______, I ______ ______ ______
______ ______ YOU ______ ______ ______ ______ ______ ______
______ ______ ______ BACK
TO ______ ______ ______ ______ ______ ______ ______ ______
______, ______ ______ ______ ______, ______, ______ ______
______ ______ ______ ______ ______ ______ ______, ______

[______-